In [1]:
from ocr import BEEStarLabelOCR

In [1]:
import os
import torch
import cv2
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')
from time import sleep

In [2]:
model1 = torch.hub.load('ultralytics/yolov5', 'custom', path = 'star_label.pt', device = 'mps')
model1.conf = 0.45

model2 = torch.hub.load('ultralytics/yolov5', 'custom', path = 'star_label2.pt', device = 'mps')
model2.conf = 0.15

Using cache found in /Users/abhishekomi/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-6-14 Python-3.8.16 torch-2.0.0 MPS

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 
Using cache found in /Users/abhishekomi/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-6-14 Python-3.8.16 torch-2.0.0 MPS

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [3]:
def detect_bee_image(img_path : str):
    def calculate_area(x1, y1, x2, y2):
        w = int(x2 - x1)
        h = int(y2 - y1)
        area = w * h
        return area

    result1 = model1(img_path)
    result2 = model2(img_path)
    out1 = result1.pandas().xyxy[0]
    out2 = result2.pandas().xyxy[0]
    if (out1.shape[0] > 0) or (out2.shape[0] > 0):
        return True
    else:
        return False

In [4]:
reports_path = 'reports_new2/'
os.listdir(reports_path)

['.DS_Store',
 'Colour_TV',
 'Refrigerator_frost_free',
 'LED_lamps',
 'Ceiling_fan',
 'AC_variable_speed',
 'AC_fixed_speed',
 'tabular_florescent_lamps',
 'Refrigerator_direct_cool',
 'Electric_water_heater']

In [5]:
# working on ac_variable_speed
working_path = reports_path + 'tabular_florescent_lamps/'
data_points = glob(working_path + '/croma/images/*')
print("total data points : ", len(data_points))

total data points :  12


In [6]:
data = pd.read_csv(working_path + '/croma/tfl_report.csv')

In [7]:
data.head(2)

,Sno,Website,Website type,Appliance,Brand,...,"What is the EE parameter mentioned in the Product Description or About or Overview section in the product page?\n\nReliance - Description , Amazon - From The manufacturer , Croma - Overview , Snapdeal - Description , Flipkart - Product Description","Is Star Rating provided in the Product Information table?\n(Yes/No)\n\nReliance - Specifications , Amazon - Product information , Croma - Specifications , Snapdeal - Other Specifications , Flipkart - Specifications","What is the Star Rating provided in the Product Information table?\n\nReliance - Specifications , Amazon - Product information , Croma - Specifications , Snapdeal - Other Specifications , Flipkart - Specifications","Is EE parameter and its value mentioned in the Product Information table?\n(Yes/No)\n\nReliance - Specifications , Amazon - Product information , Croma - Specifications , Snapdeal - Other Specifications , Flipkart - Specifications","What is the EE parameter and its value mentioned in the Product Information table?\n\nReliance - Specifications , Amazon - Product information , Croma - Specifications , Snapdeal - Other Specifications , Flipkart - Specifications"
0,0,croma,E- commerce website,tabular_florescent_lamps,HIFFIN,...,NaN,No,NaN,No,NaN
1,1,croma,E- commerce website,tabular_florescent_lamps,Digitek,...,NaN,No,NaN,No,NaN


In [8]:
data['image_to_bee_label_threshold'] = ['NA' for _ in range(data.shape[0])]

In [14]:
ocr = BEEStarLabelOCR()

for data_point in tqdm(data_points):
    try:
        point = int(data_point.split('/')[-1])
        images = glob(data_point + '/*')
        
        bee_count = 0
        all_image_report = []
        readable = 'No'
        for img in images:
            is_bee = detect_bee_image(img)
            if is_bee:
                bee_count += 1
                report_ocr, area = ocr.run_ocr(img, "AC")
                if report_ocr == None:
                    continue
                else:
                    all_image_report.append((report_ocr, area))
        
        if bee_count > 0:
            # print(point)
            if len(all_image_report) > 0:
                report_ocr, area = sorted(all_image_report, key = lambda x: x[1], reverse = True)[0]
                data['image_to_bee_label_threshold'][point] = round(area, 2)
                if area > 20:
                    readable = 'Yes'
                    data['Does the product page contain any “Picture of BEE Star Label” as one of the pictures available for the appliance model?\n(Yes/No)'][point] = 'Yes'
                    data['How many pictures of BEE Star Label are shown for appliance model on the product page?'][point] = bee_count
                    data['Is the information displayed on the “Picture of BEE Star Label” readable?\n\n(Yes/No)'][point] = readable
                    data['a)\xa0\xa0\xa0\xa0 ISEER'][point] = report_ocr['iseer']
                    data['b)\xa0\xa0\xa0\xa0 Label period'][point] = report_ocr['label_period']
                    data['c)\xa0\xa0\xa0\xa0 Appliance/Type'][point] = report_ocr['appliance_type']
                    data['d)\xa0\xa0\xa0\xa0 Model/Year'][point] = report_ocr['model_year']
                    data['e)\xa0\xa0\xa0\xa0 Cooling Capacity (100%)'][point] = report_ocr['cooling_100']
                    data['f)\xa0\xa0\xa0\xa0\xa0 Cooling Capacity (50%)'][point] = report_ocr['cooling_50']
                    data['g)\xa0\xa0\xa0\xa0 Electricity consumption'][point] = report_ocr['electricity_consumption']
                    data['h)\xa0\xa0\xa0\xa0 Compressor Type'][point] = report_ocr['compressor']    
                    data['i)\xa0\xa0\xa0\xa0\xa0\xa0 Heat Pump'][point] = report_ocr['heat_pump']

                    # data['a)\xa0\xa0\xa0\xa0 Electricity Consumption (units per year)'][point] = report_ocr['electricity_consumption']
                    # data['b)\xa0\xa0\xa0\xa0 Label period'][point] = report_ocr['label_period']
                    # data['c)\xa0\xa0\xa0\xa0 Appliance'][point] = report_ocr['appliance_type']
                    # data['d)\xa0\xa0\xa0\xa0 Brand'][point] = report_ocr['brand']
                    # data['e)\xa0\xa0\xa0\xa0 Model Number/Name'][point] = report_ocr['model_year']
                    # data['f)\xa0\xa0\xa0\xa0\xa0 Type '][point] = report_ocr['type']
                    # data['g)\xa0\xa0\xa0\xa0 Total Volume (litre)'][point] = report_ocr['total_volume']
                else:
                    data['Does the product page contain any “Picture of BEE Star Label” as one of the pictures available for the appliance model?\n(Yes/No)'][point] = 'Yes'
                    data['How many pictures of BEE Star Label are shown for appliance model on the product page?'][point] = bee_count
                    data['Is the information displayed on the “Picture of BEE Star Label” readable?\n\n(Yes/No)'][point] = readable
            else:
                data['image_to_bee_label_threshold'][point] = 'not detectable (too small size)'
                data['Does the product page contain any “Picture of BEE Star Label” as one of the pictures available for the appliance model?\n(Yes/No)'][point] = 'Yes'
                data['How many pictures of BEE Star Label are shown for appliance model on the product page?'][point] = bee_count
                data['Is the information displayed on the “Picture of BEE Star Label” readable?\n\n(Yes/No)'][point] = readable
        else:
            data['Does the product page contain any “Picture of BEE Star Label” as one of the pictures available for the appliance model?\n(Yes/No)'][point] = 'No'
            data['How many pictures of BEE Star Label are shown for appliance model on the product page?'][point] = 0
            data['Is the information displayed on the “Picture of BEE Star Label” readable?\n\n(Yes/No)'][point] = 'No'

        data['Is Star Rating mentioned in the Product Description or About or Overview section in the product page?\n(Yes/No)\n\nReliance - Description , Amazon - From The manufacturer , Croma - Overview , Snapdeal - Description , Flipkart - Product Description '][point] = 'No'
        data['What is the Star Rating mentioned in the Product Description or About or Overview section in the product page?\n\nReliance - Description , Amazon - From The manufacturer , Croma - Overview , Snapdeal - Description , Flipkart - Product Description '][point] = 'NA'
        data['Is EE parameter mentioned in the Product Description or About or Overview section in the product page?\n(Yes/No)\n\nReliance - Description , Amazon - From The manufacturer , Croma - Overview , Snapdeal - Description , Flipkart - Product Description '][point] = 'No'
        data['What is the EE parameter mentioned in the Product Description or About or Overview section in the product page?\n\nReliance - Description , Amazon - From The manufacturer , Croma - Overview , Snapdeal - Description , Flipkart - Product Description '][point] = 'NA'

    except Exception as e:
        print(str(e))

data = data.fillna('NA')
data.to_csv('test_reports/air_conditioner_vs.csv')

Using CPU. Note: This module is much faster with a GPU.
100%|██████████| 273/273 [16:01<00:00,  3.52s/it] 


In [15]:
data.iloc[:5, 6:15]

,Web Link of product page,Does the product page contain any “Picture of BEE Star Label” as one of the pictures available for the appliance model?\n(Yes/No),How many pictures of BEE Star Label are shown for appliance model on the product page?,Is the information displayed on the “Picture of BEE Star Label” readable?\n\n(Yes/No),a) Electricity Consumption (units per year),b) Label period,c) Appliance,d) Brand,e) Model Number/Name
0,https://www.croma.com/croma-165-litres-2-star-...,Yes,3.0,Yes,"194""",NA,NA,NA,NA
1,https://www.croma.com/croma-50-litres-2-star-d...,Yes,4.0,No,NA,NA,NA,NA,NA
2,https://www.croma.com/croma-185-litres-2-star-...,Yes,3.0,Yes,NA,NA,NA,NA,NA
3,https://www.croma.com/candy-225-liters-2-star-...,No,0.0,No,NA,NA,NA,NA,NA
4,https://www.croma.com/haier-185-litres-2-star-...,Yes,1.0,No,NA,NA,NA,NA,NA


In [16]:
data.to_csv('test_reports/refrigerator_direct_cool2.csv')

In [9]:
for data_point in tqdm(data_points):
    point = int(data_point.split('/')[-1])
    images = glob(data_point + '/*')
    
    bee_count = 0
    for img in images:
        try:
            is_bee = detect_bee_image(img)
        except:
            is_bee = False
        if is_bee:
            bee_count += 1

    if bee_count > 0:
        data['Does the product page contain any “Picture of BEE Star Label” as one of the pictures available for the appliance model?\n(Yes/No)'][point] = 'Yes'
        data['How many pictures of BEE Star Label are shown for appliance model on the product page?'][point] = bee_count
        data['Is the information displayed on the “Picture of BEE Star Label” readable?\n\n(Yes/No)'][point] = 'No'
    else:
        data['Does the product page contain any “Picture of BEE Star Label” as one of the pictures available for the appliance model?\n(Yes/No)'][point] = 'No'
        data['How many pictures of BEE Star Label are shown for appliance model on the product page?'][point] = bee_count
        data['Is the information displayed on the “Picture of BEE Star Label” readable?\n\n(Yes/No)'][point] = 'No'


100%|██████████| 12/12 [00:04<00:00,  2.94it/s]


In [25]:
data.shape

(600, 35)

In [25]:

data = data.fillna('NA')
data.to_csv('test_reports/tfl_report.csv')

In [12]:
detect_bee_image(reports_path + 'Ac_variable_speed/amazon/images/213/3.png')

         xmin        ymin         xmax         ymax  confidence  class  \
0  809.358765  226.852722  1467.440063  1478.105713    0.616284      0   
1  380.046722  864.392029   542.029480  1164.708496    0.615953      0   

       name  
0  bee_star  
1  bee_star  
         xmin        ymin         xmax         ymax  confidence  class  name
0  378.350677  864.076965   544.268982  1009.937805    0.484402      0  star
1  814.452942  251.303162  1467.279419   848.704468    0.167733      0  star


True

In [3]:
ocr = BEEStarLabelOCR()

Using CPU. Note: This module is much faster with a GPU.


In [14]:
print(ocr.run_ocr(reports_path + 'Ac_variable_speed/amazon/images/213/3.png', "AC"))

({'iseer': '3.96*', 'label_period': None, 'appliance_type': None, 'model_year': None, 'cooling_100': None, 'cooling_50': None, 'electricity_consumption': None, 'compressor': None, 'heat_pump': 'No'}, 9.0)


In [12]:
data.iloc[[307]].values

array([[307, 'Amazon', 'E- commerce website', 'Air Conditioner', 'LG', 'RS-Q19ENZE', 'https://www.amazon.in//LG-Convertible-Anti-Virus-Protection-RS-Q19ENZE/dp/B0C593V9JF/ref=sr_1_288?crid=3F7OTI1W11SOG&keywords=Air+Conditioners+Inverter&qid=1685191620&sprefix=%2Caps%2C533&sr=8-288', 'Yes', 1, 'Yes', '5.20',
        'iat july 2022 to 31st Dec 2024', 'RACISplit', 'RS-QI9ENZEI2023', None, None, None, 'Variable Specd', 'No', 'Yes', 5.0, 'No', 'Yes', 5.0, 'Yes', 5.2, 'pending', 'pending', 'pending', 'pending', 'No', nan, 'No', nan]], dtype=object)

In [24]:
data.columns[10]

'a)\xa0\xa0\xa0\xa0 Electricity Consumption (units per year)'

In [ ]:
data['Is the information displayed on the “Picture of BEE Star Label” readable?\n\n(Yes/No)'][point]
data['b)\xa0\xa0\xa0\xa0 Label period'][point]
data['c)\xa0\xa0\xa0\xa0 Appliance'][point]
data['d)\xa0\xa0\xa0\xa0 Brand'][point]
data['e)\xa0\xa0\xa0\xa0 Model Number/Name'][point]
data['f)\xa0\xa0\xa0\xa0\xa0 Type '][point]
data['g)\xa0\xa0\xa0\xa0 Total Volume (litre)'][point]

nan